In [264]:
# Import the Pandas library and read and merge the CSV files to create a comprehensive DataFrame.
import pandas as pd
merged_df = pd.read_csv('https://raw.githubusercontent.com/GBov81/pandas-challenge/main/schools_complete.csv')\
    .merge(pd.read_csv('https://raw.githubusercontent.com/GBov81/pandas-challenge/main/students_complete.csv'), on='school_name')

# Calculate the total number of unique schools in the district.
total_schools = merged_df['school_name'].nunique()

# Calculate the total number of students and the combined budget for all schools.
total_students = merged_df['Student ID'].nunique()
total_budget = merged_df['budget'].unique().sum()

# Calculate the average math and reading scores across all students and schools.
average_math_score = merged_df['math_score'].mean()
average_reading_score = merged_df['reading_score'].mean()

# Calculate the percentage of students passing math and reading individually.
percent_passing_math = (merged_df['math_score'] > 70).mean() * 100
percent_passing_reading = (merged_df['reading_score'] > 70).mean() * 100

# Calculate the percentage of students passing both subjects.
percent_overall_passing = ((merged_df['math_score'] > 70) & (merged_df['reading_score'] > 70)).mean() * 100

# Create and print a DataFrame for district-level summary metrics.
district_summary_data = {
    'Total Schools': [total_schools],
    'Total Students': [total_students],
    'Total Budget': [total_budget],
    'Average Math Score': [average_math_score],
    'Average Reading Score': [average_reading_score],
    '% Passing Math': [percent_passing_math],
    '% Passing Reading': [percent_passing_reading],
    '% Overall Passing': [percent_overall_passing]}
district_summary_df = pd.DataFrame(district_summary_data)
print(district_summary_df)

   Total Schools  Total Students  Total Budget  Average Math Score  \
0             15           39170      24649428           78.985371   

   Average Reading Score  % Passing Math  % Passing Reading  % Overall Passing  
0               81.87784       72.392137          82.971662          60.801634  


In [265]:
# Group the merged dataset by school name and type, then calculate necessary metrics.
school_summary_df = merged_df.groupby(['school_name', 'type']).agg({
    'Student ID': 'count',
    'budget': 'mean',
    'math_score': 'mean',
    'reading_score': 'mean'}).reset_index()

# Calculate the passing percentages for math, reading, and overall passing for each school.
passing_math = (merged_df['math_score'] >= 70).groupby([merged_df['school_name'], merged_df['type']]).mean() * 100
passing_reading = (merged_df['reading_score'] >= 70).groupby([merged_df['school_name'], merged_df['type']]).mean() * 100
passing_overall = ((merged_df['math_score'] >= 70) & (merged_df['reading_score'] >= 70)).groupby([merged_df['school_name'], merged_df['type']]).mean() * 100

# Assign the calculated passing percentages to the school_summary_df with renamed columns.
school_summary_df['% Passing Math'] = passing_math.values
school_summary_df['% Passing Reading'] = passing_reading.values
school_summary_df['% Overall Passing'] = passing_overall.values

# Calculate the per student budget for each school.
school_summary_df['Per Student Budget'] = school_summary_df['budget'] / school_summary_df['Student ID']

# Rename columns for better clarity and print.
school_summary_df.rename(columns={
    'school_name': 'School Name',
    'type': 'School Type',
    'Student ID': 'Total Students',
    'budget': 'Total School Budget',
    'math_score': 'Average Math Score',
    'reading_score': 'Average Reading Score'}, inplace=True)
print(school_summary_df)

              School Name School Type  Total Students  Total School Budget  \
0      Bailey High School    District            4976            3124928.0   
1     Cabrera High School     Charter            1858            1081356.0   
2    Figueroa High School    District            2949            1884411.0   
3        Ford High School    District            2739            1763916.0   
4     Griffin High School     Charter            1468             917500.0   
5   Hernandez High School    District            4635            3022020.0   
6      Holden High School     Charter             427             248087.0   
7       Huang High School    District            2917            1910635.0   
8     Johnson High School    District            4761            3094650.0   
9        Pena High School     Charter             962             585858.0   
10  Rodriguez High School    District            3999            2547363.0   
11    Shelton High School     Charter            1761           

In [266]:
# Identify the top 5 performing schools based on % Overall Passing and store them in 'top_schools'.
top_schools = school_summary_df.sort_values(by="% Overall Passing", ascending=False).head(5)

In [267]:
# Identify the bottom 5 performing schools based on % Overall Passing and store them in 'bottom_schools'.
bottom_schools = school_summary_df.sort_values(by="% Overall Passing").head(5)

In [268]:
# Calculate average math scores by grade for each school using a pivot table.
math_scores_by_grade = merged_df.pivot_table(values='math_score', index='school_name', columns='grade', aggfunc='mean')

In [269]:
# Calculate average reading scores by grade for each school using a pivot table.
reading_scores_by_grade = merged_df.pivot_table(values='reading_score', index='school_name', columns='grade', aggfunc='mean')

In [270]:
# Define spending bins and labels for school spending categories.
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Categorize schools into spending ranges based on per student budget.
school_summary_df['Spending Ranges (Per Student)'] = pd.cut(school_summary_df['Per Student Budget'], spending_bins, labels=spending_labels)

# Group schools by spending range and calculate the mean scores and passing percentages.
spending_summary = school_summary_df.groupby('Spending Ranges (Per Student)').agg({
    "Average Math Score": "mean",
    "Average Reading Score": "mean",
    "% Passing Math": "mean",
    "% Passing Reading": "mean",
    "% Overall Passing": "mean"})

In [271]:
# Define bins and labels for school size categories.
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Categorize schools based on their total student count into size ranges.
school_summary_df['School Size'] = pd.cut(school_summary_df['Total Students'], size_bins, labels=size_labels)

# Group schools by size category and calculate the mean scores and passing percentages.
size_summary = school_summary_df.groupby('School Size').agg({
    "Average Math Score": "mean",
    "Average Reading Score": "mean",
    "% Passing Math": "mean",
    "% Passing Reading": "mean",
    "% Overall Passing": "mean"})